<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-anomaly-detection/notebooks/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 異常検知（Anomaly detection）
標準的な状態や想定から逸脱しているデータや事象を特定する技術


In [3]:
import numpy as np
import pandas as pd

## 正規分布に従うデータの異常検知
### 例
製品の温度をセンサーで監視している場合の異常検知
<br>
<br>
### 手法

#### 3$\sigma$法
データが正規分布に従うと仮定し、平均値から標準偏差の3倍以上離れた値を異常値とみなす方法
<br>
#### マハラノビス・タグチ法
データの各次元間の相関を考慮した距離尺度を用いて、平均値から離れた値を異常値とみなす方法
<br>
#### ホテリングの$T^2$法
マハラノビス距離を拡張した手法で、データの平均値からのずれを検定統計量として用いて異常値を検出する方法
<br>
#### 密度比推定
正常データと異常データの確率密度比を推定することで異常を検知する手法
<br>

In [4]:
from scipy.spatial.distance import mahalanobis
from scipy.stats import f
from sklearn.neighbors import KernelDensity


### 3σ法
# threshold: データの平均値から標準偏差の何倍離れたら異常値とみなすか
def three_sigma_method(data, threshold=3):
  mean = np.mean(data)
  std = np.std(data)
  lower_bound = mean - threshold * std
  upper_bound = mean + threshold * std
  if data.ndim == 1:
    anomaly_indices = np.where((data < lower_bound) | (data > upper_bound))[0]
  elif data.ndim >= 2:
    anomaly_indices = np.where(np.any((data < lower_bound) | (data > upper_bound), axis=1))[0]
  return data[anomaly_indices]

### マハラノビス距離
# マハラノビス距離: データの相関を考慮した距離尺度
# threshold_percentile: マハラノビス距離の分布におけるパーセンタイル値を超える
#                       距離を持つデータを異常値とみなす
def mahalanobis_distance_method(data, threshold_percentile=99):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    std = np.std(data)

    # 標準偏差が0の場合、ゼロ除算を避けるために微小な値を加算
    if std == 0:
      std = 1e-6

    distances = np.abs((data - mean) / std)

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomalies = data[distances > threshold]
  elif data.ndim >= 2:
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)

    distances = [mahalanobis(x, mean, inv_cov) for x in data]

    # 異常値とみなす閾値を設定
    threshold = np.percentile(distances, threshold_percentile)

    # 閾値を超えるデータを異常値として検出
    anomaly_indices = np.where(np.array(distances) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies

### ホテリングのT^2法
def hotelling_t2_method(data):
  mean = np.mean(data, axis=0)
  if data.ndim == 1:
    var = np.var(data)
    t2_values = [(x - mean)**2 / var for x in data]
    # f.ppf: F分布のパーセント点関数
    threshold = f.ppf(0.95, 1, len(data) - 1)  # 異常値とみなす閾値を設定
    anomalies = [data[i] for i, t2 in enumerate(t2_values) if t2 > threshold]
  elif data.ndim >= 2:
    mean = np.mean(data, axis=0)
    cov = np.cov(data, rowvar=False)
    inv_cov = np.linalg.inv(cov)
    t2_values = [mahalanobis(x, mean, inv_cov)**2 for x in data]
    threshold = f.ppf(0.95, data.shape[1], data.shape[0] - data.shape[1] - 1)  # 異常値とみなす閾値を設定
    anomaly_indices = np.where(np.array(t2_values) > threshold)[0]
    anomalies = data[anomaly_indices]
  return anomalies


# 正常データ生成
normal_data = np.random.normal(loc=25, scale=2, size=100)

# 異常データ生成
anomaly_data = np.random.normal(loc=25, scale=2, size=100)
# 異常値の追加
anomaly_indices = [10, 20, 30]  # 異常値のインデックス
anomaly_values = [35, 15, 32]  # 異常値
anomaly_data[anomaly_indices] = anomaly_values

# 結果の出力
anomalies = three_sigma_method(anomaly_data)
print("================== 3σ法 =================")
print("異常値:", anomalies)
print("\n")

anomalies = mahalanobis_distance_method(anomaly_data)
print("===== マハラノビス距離による異常検知 =====")
print("異常値:", anomalies)
print("\n")

anomalies = hotelling_t2_method(anomaly_data)
print("===== ホテリングのT^2法による異常検知 ====")
print("異常値:", anomalies)
print("\n")


================== 3σ法 =================
異常値: [35. 15.]


===== マハラノビス距離による異常検知 =====
異常値: [35.]


===== ホテリングのT^2法による異常検知 ====
異常値: [np.float64(35.0), np.float64(15.0), np.float64(32.0), np.float64(30.345462268832556)]




## 非正規データの異常検知
### 例
ウェブサイトへのアクセス数の異常検知
<br>
<br>
### 手法

#### ガンマ分布の当てはめ
データがガンマ分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。データがガンマ分布に従う場合、高い精度で異常を検出できるが、従わない場合、精度が低下する。
<br>

#### カイ二乗分布への当てはめ
データがカイ二乗分布に従うと仮定し、その分布から大きく外れた値を異常値とみなす方法。主に、特徴量の値が正の値を取り、歪んだ分布をしている場合に有効。<br>
<br>

#### $k$近傍法
各データポイントからk番目に近いデータポイントまでの距離を計算し、距離が大きいデータポイントを異常値と判定するアルゴリズム。$k$はデータセットのサイズの平方根よりも小さい奇数にすると良い。
<br>

#### $k$ means法
データを複数のクラスタに分割し、どのクラスタにも属さないデータポイントを異常値と判定するアルゴリズム。$k$を推定する手法としてエルボー法やシルエット分析などがある。
<br>

#### 混合ガウス分布モデル（Gaussian Mixture Model, GMM）
データが複数のガウス分布（正規分布）の混合で表現できると仮定し、低確率なデータ点を異常値とみなす方法。
<br>

#### One-Class SVM
正常データのみを用いて、正常データの領域を学習するアルゴリズム。 学習した領域から外れたデータは異常値と判定する。
<br>

#### [密度比推定](#scrollTo=1dE3Gampih0k&line=1&uniqifier=1)
<br>

#### 孤立フォレスト（Isolation Forest）
データポイントをランダムに分割していくことで、異常値を孤立させるアルゴリズム。
<br>

#### Local Outlier Factor (LOF)
データポイントの局所的な密度を計算し、密度が低いデータポイントを異常値と判定するアルゴリズム。
<br>



In [5]:
import scipy.stats as stats
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import IsolationForest


### ガンマ分布の当てはめ
def fitting_gamma_distribution(data):
  # データにガンマ分布を当てはめる
  shape, loc, scale = stats.gamma.fit(data)

  # 異常度を計算する
  anomaly_scores = 1 / stats.gamma.pdf(data, shape, loc, scale)

  # 閾値を設定する
  threshold = np.mean(anomaly_scores) + 3 * np.std(anomaly_scores)

  # 異常値を検出する
  anomalies = data[anomaly_scores > threshold]

  return anomalies


### k近傍法
def k_nearest_neighbors(data, k):
  data = data.reshape(-1, 1)

  # k近傍の計算
  knn = NearestNeighbors(n_neighbors=k)
  knn.fit(data)
  distances, indices = knn.kneighbors(data)

  # 異常度の算出
  anomaly_score = distances[:, -1]
  threshold = np.percentile(anomaly_score, 99)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score > threshold)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


### 孤立フォレスト
def isolation_forest(data):
  data = data.reshape(-1, 1)

  # モデルの学習
  model = IsolationForest()
  model.fit(data)

  # 異常度の算出
  anomaly_score = model.decision_function(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < -0.35)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


def power_law_distribution(alpha, xmin, xmax, size):
    """べき乗則に従う乱数を生成する関数

    Args:
        alpha (float): べき乗則の指数
        xmin (float): 最小値
        xmax (float): 最大値
        size (int): 生成する乱数の数

    Returns:
        numpy.ndarray: べき乗則に従う乱数
    """

    u = np.random.uniform(size=size)
    return (xmin ** (1 - alpha) + (xmax ** (1 - alpha) - xmin ** (1 - alpha)) * u) ** (1 / (1 - alpha))


# 正常データの生成
normal_data = power_law_distribution(alpha=2, xmin=1, xmax=100, size=1000)

# 異常データの生成
outliers = np.random.uniform(low=2*normal_data.max(), high=3*normal_data.max(), size=10)
anomaly_data = np.concatenate([normal_data, outliers])

all_data = np.concatenate([normal_data, anomaly_data])

#### ガンマ分布の当てはめ
anomalies = fitting_gamma_distribution(all_data)
print("============ ガンマ分布の当てはめ ==========")
print("異常値:", anomalies)
print("\n")

#### k近傍法
anomalies = k_nearest_neighbors(all_data, 5)
print("================== k近傍法 =================")
print("異常値:", anomalies)
print("\n")

#### 孤立フォレスト (Isolation Forest)
anomalies = isolation_forest(all_data)
print("=============== 孤立フォレスト ==============")
print("異常値:", anomalies)
print("\n")


============ ガンマ分布の当てはめ ==========
異常値: [268.93160448 290.18407073 284.94866004 276.45041134 289.92953617
 273.38715601 273.10970905]


================== k近傍法 =================
異常値: [[ 96.74146167]
 [ 70.27543975]
 [ 66.48558792]
 [ 78.56197391]
 [ 88.67405235]
 [ 96.74146167]
 [ 70.27543975]
 [ 66.48558792]
 [ 78.56197391]
 [ 88.67405235]
 [268.93160448]
 [290.18407073]
 [242.3174822 ]
 [284.94866004]
 [276.45041134]
 [226.79897513]
 [249.05686843]
 [289.92953617]
 [273.38715601]
 [273.10970905]]


=============== 孤立フォレスト ==============
異常値: [[268.93160448]
 [290.18407073]
 [242.3174822 ]
 [284.94866004]
 [276.45041134]
 [226.79897513]
 [249.05686843]
 [289.92953617]
 [273.38715601]
 [273.10970905]]




## 不要次元のある次元データの異常検知
### 例
顧客の購買データを用いた異常検知
<br>
<br>
### 次元削減を用いる手法

#### 主成分分析（PCA）
データの分散を最大化するように新しい軸を定義し、その軸にデータを射影することで次元を削減する手法。異常なデータ点は、主成分空間において、正常なデータ点から離れた位置に配置される傾向がある。
<br>

#### 確率的主成分分析（Probabilistic PCA）
PCAを確率モデルとして拡張した手法。データにノイズが含まれる場合に、よりロバストな結果が得られる。（＝データにノイズや外れ値が含まれていても、分析結果が大きく影響を受けにくい）
<br>

#### カーネル主成分分析（Kernel PCA）
非線形な関係を持つデータに対して、カーネル関数を使用して高次元空間に写像し、その空間でPCAを行うことで次元を削減する手法。
<br>

#### 因子分析（Factor Analysis）
観測変数の背後にある潜在変数を推定し、それらを用いて次元を削減する手法。
<br>

#### 独立成分分析（ICA）
観測変数を、統計的に独立な成分に分解することで次元を削減する手法。
<br>

#### t-distributed Stochastic Neighbor Embedding (t-SNE)
高次元データを低次元空間に埋め込む際に、データの局所的な構造を保持する様に設計された手法。異常検知では、正常なデータ点と異常なデータ点が、低次元空間において明確に分離されることが期待する。
<br>

#### Uniform Manifold Approximation and Projection (UMAP)
高次元データを低次元空間に埋め込む際に、データのトポロジカルな構造を保持する様に設計された手法です。t-SNEと同様に、異常検知では、正常なデータ点と異常なデータ点が、低次元空間において明確に分離されることを期待する。
<br>

### 特徴量選択を用いる手法
#### フィルター法
各特徴量と目的変数の間の相関や相互情報量などを用いて、重要度の低い特徴量を除去する手法。
<br>

#### ラッパー法
特徴量のサブセットを選択し、そのサブセットを用いて学習したモデルの性能を評価することで、最適な特徴量を選択する手法。
<br>

#### 埋め込み法
モデルの学習過程で特徴量選択を行う手法。LASSO回帰や決定木などが用いられる。
<br>

### その他の手法
#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [孤立フォレスト（Isolation Forest）](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [Local Outlier Factor (LOF)](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>






In [12]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import GridSearchCV


### One-Class SVM
def one_class_svm(data):
  data = data.reshape(-1, 1)

  # モデルの学習
  model = OneClassSVM()

  # 閾値の候補を指定
  param_grid = {'nu': [0.1, 0.2, 0.3], 'gamma': ['scale', 'auto']}

  # 交差検定を実行
  grid_search = GridSearchCV(model, param_grid, scoring='f1')
  grid_search.fit(data)

  # 最適なモデルを取得
  model = grid_search.best_estimator_

  # 最適な閾値を取得
  best_threshold = grid_search.best_estimator_.decision_function(data).min()

  # 異常度の算出
  anomaly_score = model.decision_function(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score < -850)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


### Local Outlier Factor (LOF)
def local_outlier_factor(data):
  data = data.reshape(-1, 1)

  # モデルの学習
  model = LocalOutlierFactor()
  anomaly_score = model.fit_predict(data)

  # 異常値の判定
  anomaly_index = np.where(anomaly_score == -1)

  # 異常値を検出する
  anomalies = data[anomaly_index]

  return anomalies


#### 正常データの生成
# 顧客ID、年齢、性別、購入金額、購入頻度、不要な次元（ランダムな数値）を生成
num_customers = 100  # 顧客数
customer_ids = np.arange(1, num_customers + 1)
ages = np.random.randint(20, 60, size=num_customers)
genders = np.random.choice(['Male', 'Female'], size=num_customers)
purchase_amounts = np.random.randint(100, 10000, size=num_customers)
purchase_frequencies = np.random.randint(1, 10, size=num_customers)
unnecessary_dimension = np.random.rand(num_customers)

# データフレームを作成
normal_data = pd.DataFrame({
  'CustomerID': customer_ids,
  'Age': ages,
  'Gender': genders,
  'PurchaseAmount': purchase_amounts,
  'PurchaseFrequency': purchase_frequencies,
  'UnnecessaryDimension': unnecessary_dimension  # 不要な次元
})

#### 異常データの生成
anomaly_data = normal_data.copy()

# 異常値を挿入するインデックスをランダムに選択
# 例：顧客の10%を異常値とする
anomaly_indices = np.random.choice(anomaly_data.index, size=int(num_customers * 0.1), replace=False)

# 選択したインデックスのデータに異常値を代入
# 例：PurchaseAmountを極端に大きくする
anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] = anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] * np.random.uniform(100, 200, size=len(anomaly_indices))


#### One-Class SVM
anomalies = one_class_svm(all_data)
print("=============== One-Class SVM ==============")
print("異常値:", anomalies)
print("\n")

# #### 孤立フォレスト（Isolation Forest）
# anomalies = isolation_forest(all_data)
# print("=============== 孤立フォレスト ==============")
# print("異常値:", anomalies)
# print("\n")

# #### Local Outlier Factor (LOF)
# anomalies = local_outlier_factor(all_data)
# print("============ Local Outlier Factor ===========")
# print("異常値:", anomalies)
# print("\n")



<ipython-input-12-d4ec15b9e6dd>:84: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[ 924781.17190045 1253254.68864096  836564.92746946  430832.86975697
  766079.46257722  460052.14903901  750831.57450826  423639.83483709
  899874.63539881  220424.29516352]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] = anomaly_data.loc[anomaly_indices, 'PurchaseAmount'] * np.random.uniform(100, 200, size=len(anomaly_indices))
/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 947, in _score
    scores = scorer(estimator, X_test, **score_params)
      

=============== One-Class SVM ==============
異常値: []


=============== 孤立フォレスト ==============
異常値: []


============ Local Outlier Factor ===========
異常値: []




## 入出力関係のあるデータの異常検知
### 例
製造工程における品質管理
<br>

### 手法

#### 線形回帰モデル
正常データを用いて、入力と出力の関係を回帰モデルで学習し、新しいデータが入力された際に、回帰モデルの予測値と実際の出力値の差が大きい場合、異常と判定する。
<br>

#### リッジ回帰
線形回帰モデルの一種で、過学習を防ぐために正則化項を導入したもの。
<br>

#### ガウス過程回帰
非線形な回帰問題を解くための機械学習手法。観測データから入力と出力の関係を確率分布として学習する。この確率分布はガウス過程（任意の有限個の点における確率変数の集合が、常に多変量正規分布に従うような確率過程）で表現され、予測値だけでなく、予測値の不確実性も推定することができる。
<br>

#### 状態空間モデル
入出力関係を状態空間モデルで表現し、観測値と予測値の差を異常度として用いる方法。
<br>

#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

### 線形回帰
def linear_regression(input_data, output_data):
  # 線形回帰モデルを学習
  model = LinearRegression()
  model.fit(input_data, output_data)

  # 予測値と実際の出力値の差を計算
  predicted_output = model.predict(input_data)
  residuals = output_data - predicted_output

  # 異常値を判定 (例：残差が大きい値を異常値とする)
  threshold = np.mean(residuals) + 2 * np.std(residuals)  # 閾値は調整が必要
  anomalies = input_data[np.abs(residuals) > threshold]

  return anomalies


### リッジ回帰
def ridge_regression(input_data, output_data):
  # リッジ回帰モデルを学習
  model = Ridge(alpha=1.0)  # alphaは正則化の強さを指定
  model.fit(input_data, output_data)

  # 予測値と実際の出力値の差を計算
  predicted_output = model.predict(input_data)
  residuals = output_data - predicted_output

  # 異常値を判定 (例：残差が大きい値を異常値とする)
  threshold = np.mean(residuals) + 2 * np.std(residuals)  # 閾値は調整が必要
  anomalies = input_data[np.abs(residuals) > threshold]

  return anomalies


# from sklearn.svm import OneClassSVM

# # One-Class SVM
#    # 入力データと出力データを結合
#    data = np.concatenate([input_data, output_data.reshape(-1, 1)], axis=1)

#    # One-Class SVMモデルを学習
#    model = OneClassSVM(nu=0.1)  # nuは異常値の割合を指定
#    model.fit(data)

#    # 異常度を算出
#    anomaly_scores = model.decision_function(data)

#    # 異常値を判定 (例：異常度が負の値を異常値とする)
#    anomalies = data[anomaly_scores < 0]


# 入力データと出力データの例
input_data = np.array([[10, 20], [12, 22], [13, 23], [11, 21], [12, 23],
                       [11, 22], [13, 24], [10, 21], [5, 10], [12, 22]])  # [5, 10] が異常値
output_data = np.array([30, 34, 36, 32, 35, 33, 37, 31, 15, 34])
all_data = np.concatenate([input_data, output_data.reshape(-1, 1)], axis=1)

#### 線形回帰
anomalies = linear_regression(input_data, output_data)
print("================== 線形回帰 ================")
print("異常値:", anomalies)
print("\n")

#### リッジ回帰
anomalies = ridge_regression(input_data, output_data)
print("================= リッジ回帰 ================")
print("異常値:", anomalies)
print("\n")

#### One-Class SVM
anomalies = one_class_svm(all_data)
print("=============== One-Class SVM ==============")
print("異常値:", anomalies)
print("\n")

================== 線形回帰 ================
異常値: []


================= リッジ回帰 ================
異常値: []


=============== One-Class SVM ==============
異常値: []




## 時系列データの異常検知
### 例
サーバーのCPU使用率の異常検知

### 手法

#### 自己回帰モデル
時系列データの過去の値を用いて、未来の値を予測するモデル。予測値と実測値の差が大きい場合に異常と判定する。
<br>

#### 移動平均法
一定期間のデータの平均値を計算し、その平均値から大きく外れた値を異常とみなす手法。
<br>

#### 指数平滑法
直近のデータに大きな重みを置くことで、移動平均法よりも長期的な変動に対応できる手法。
<br>

#### ARIMAモデル
時系列データの自己相関を考慮したモデルで、予測値と実測値の差が大きい場合、異常とみなす。
<br>

#### [$k$近傍法](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### サポートベクターマシン (SVM)
正常データと異常データを分離する超平面を学習する手法。[One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)は正常データのみを用いる。
<br>

#### [孤立フォレスト (Isolation Forest)](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### LSTM (Long Short-Term Memory)
<br>

#### 特異スペクトル変換法
時系列データの長期的な依存関係を学習できるニューラルネットワーク。正常な時系列データを学習し、学習データと大きく異なるパターンを持つデータを異常とみなす。
<br>

#### [状態空間モデル](#scrollTo=NWg4xZztixiP&line=23&uniqifier=1)

#### Autoencoder
データを低次元空間に圧縮し、復元するニューラルネットワーク。正常な時系列データを学習し、復元誤差が大きいデータを異常とみなす。
<br>

#### Variational Autoencoder (VAE)
Autoencoderを確率モデルとして拡張した手法。
<br>

#### Generative Adversarial Networks (GANs)
 正常な時系列データを生成する生成器と、生成されたデータが正常か異常かを判定する識別器を競合的に学習させる手法。識別器が正常と判定できないデータを異常とみなす。
 <br>

In [ ]:
# 移動平均法
import numpy as np

def moving_average_method(data, window_size):
    # 移動平均を計算
    moving_average = np.convolve(data, np.ones(window_size) / window_size, mode='valid')

    # 異常値を判定 (例：移動平均から3σ以上外れた値を異常値とする)
    threshold = 3 * np.std(data - moving_average)  # 閾値は調整が必要
    anomalies = data[np.abs(data - moving_average) > threshold]

    return anomalies


# CPU使用率データの例
cpu_usage = np.array([20, 22, 25, 23, 21, 24, 26, 28, 80, 25])  # 80 が異常値

# 異常値を検出
anomalies = moving_average_method(cpu_usage, window_size=3)

# ARIMAモデル
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

def arima_model(data, order):
    # ARIMAモデルを学習
    model = ARIMA(data, order=order)
    model_fit = model.fit()

    # 予測値と実測値の差を計算
    residuals = model_fit.resid

    # 異常値を判定 (例：残差が大きい値を異常値とする)
    threshold = 3 * np.std(residuals)  # 閾値は調整が必要
    anomalies = data[np.abs(residuals) > threshold]

    return anomalies

# CPU使用率データの例
cpu_usage = np.array([20, 22, 25, 23, 21, 24, 26, 28, 80, 25])  # 80 が異常値

# 異常値を検出
anomalies = arima_model(cpu_usage, order=(5, 1, 0))  # orderはモデルの次数を指定


# LSTM (Long Short-Term Memory)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def lstm_model(data, timesteps):
    # データをLSTMモデルの入力形式に変換
    X = []
    y = []
    for i in range(timesteps, len(data)):
        X.append(data[i - timesteps:i])
        y.append(data[i])
    X = np.array(X)
    y = np.array(y)

    # LSTMモデルを構築
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(timesteps, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # LSTMモデルを学習
    model.fit(X, y, epochs=100, batch_size=32)

    # 予測値と実測値の差を計算
    predictions = model.predict(X)
    residuals = y - predictions.flatten()

    # 異常値を判定 (例：残差が大きい値を異常値とする)
    threshold = 3 * np.std(residuals)  # 閾値は調整が必要
    anomalies = data[np.abs(residuals) > threshold]

    return anomalies

# CPU使用率データの例
cpu_usage = np.array([20, 22, 25, 23, 21, 24, 26, 28, 80, 25])  # 80 が異常値

# 異常値を検出
anomalies = lstm_model(cpu_usage, timesteps=3)  # timestepsは過去のデータを使用する期間を指定


## 変数間に関係があるデータの異常検知
### 例
クレジットカードの不正利用検知

### 手法
#### 疎構造学習
高次元データにおいて、変数間の関係が疎であることを仮定して、データの構造を学習する手法。正常なデータにおける変数間の関係を疎構造として学習し、その構造から外れるデータを異常とみなす。
<br>

#### ベイジアンネットワーク
変数間の因果関係を表現したグラフィカルモデル。変数間の条件付き確率を学習し、異常な条件付き確率を持つデータを異常とみなす。
<br>

#### マルコフ確率場
変数間の相互作用を表現したグラフィカルモデル。変数間のポテンシャル関数を学習し、異常なポテンシャル関数を持つデータを異常とみなす。
<br>

#### [k-means法](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### DBSCAN
密度ベースのクラスタリングアルゴリズム。データポイントを、高密度領域にあるデータポイントと低密度領域にあるデータポイント（ノイズ）に分類し、低密度領域にあるデータポイントを異常とみなす。
<br>

#### [One-Class SVM](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

#### [孤立フォレスト (Isolation Forest)](#scrollTo=FMOqmjNLiot5&line=7&uniqifier=1)
<br>

#### [Local Outlier Factor (LOF)](#scrollTo=FMOqmjNLiot5&line=33&uniqifier=1)
<br>

In [ ]:
# One-Class SVM
from sklearn.svm import OneClassSVM

# 取引データの例
transaction_data = [[100, 10, 1], [120, 12, 2], [130, 13, 1], [110, 11, 3], [125, 12, 2],
                    [115, 11, 1], [135, 13, 3], [105, 10, 2], [1000, 20, 5], [120, 12, 1]]  # [1000, 20, 5] が不正利用

# One-Class SVMモデルを学習
model = OneClassSVM(nu=0.1)  # nuは異常値の割合を指定
model.fit(transaction_data)

# 異常度を算出
anomaly_scores = model.decision_function(transaction_data)

# 異常値を判定 (例：異常度が負の値を不正利用と判定)
fraudulent_transactions = [transaction_data[i] for i, score in enumerate(anomaly_scores) if score < 0]

# 孤立フォレスト (Isolation Forest)
from sklearn.ensemble import IsolationForest

# 取引データの例
transaction_data = [[100, 10, 1], [120, 12, 2], [130, 13, 1], [110, 11, 3], [125, 12, 2],
                    [115, 11, 1], [135, 13, 3], [105, 10, 2], [1000, 20, 5], [120, 12, 1]]  # [1000, 20, 5] が不正利用

# Isolation Forestモデルを学習
model = IsolationForest()
model.fit(transaction_data)

# 異常度を算出
anomaly_scores = model.decision_function(transaction_data)

# 異常値を判定 (例：異常度が低い値を不正利用と判定)
fraudulent_transactions = [transaction_data[i] for i, score in enumerate(anomaly_scores) if score < -0.35]  # 閾値は調整が必要

# Local Outlier Factor (LOF)
from sklearn.neighbors import LocalOutlierFactor

# 取引データの例
transaction_data = [[100, 10, 1], [120, 12, 2], [130, 13, 1], [110, 11, 3], [125, 12, 2],
                    [115, 11, 1], [135, 13, 3], [105, 10, 2], [1000, 20, 5], [120, 12, 1]]  # [1000, 20, 5] が不正利用

# Local Outlier Factorモデルを学習
model = LocalOutlierFactor(n_neighbors=20)  # n_neighborsはkの値を指定
anomaly_scores = model.fit_predict(transaction_data)

# 異常値を判定 (例：異常度が-1の値を不正利用と判定)
fraudulent_transactions = [transaction_data[i] for i, score in enumerate(anomaly_scores) if score == -1]
